In [1]:
import time
import torch
import json
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

from tqdm import tqdm
from PIL import Image
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Subset, Dataset, DataLoader
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck, ResNet18_Weights, ResNet34_Weights, resnet18
from torchvision.datasets import ImageFolder
from utils.loss_functions import tkd_kdloss

from models_package.models import Teacher, Student
import models_package
from utils.compare_tools import compare_model_size, compare_inference_time, compare_performance_metrics, plot_comparison
from utils.misc_tools import best_LR, best_LR_wider, train_teacher, train_teacher_wider, retrieve_teacher_class_weights, new_teacher_class_weights

In [2]:
# Hyperparameters
learning_rate = 0.001379 # 0.096779
num_epochs = 2 # 200
temperature = 4.0
alpha = 0.9
momentum = 0.9
num_classes = 30
step_size = 30
gamma = 0.1
patience = 7  # for early stopping
lmda = 3
batch_size = 32
num_workers = 4

class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]
class_labels_new = torch.tensor([i for i in range(len(class_labels))])



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
len(class_labels)

30

## Test dataloader from utils

In [5]:
from data.data_loader import load_wider

train_file = ['data/wider/trainval_wider.json']
test_file = ['data/wider/test_wider.json']

class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]

trainloader, testloader  = load_wider(train_file, test_file, class_labels, batch_size, num_workers)



Compose(
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)
Compose(
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)


In [6]:
# from collections import Counter

# # Iterate through batches in the trainloader
# for batch_idx, batch in enumerate(trainloader):
#     # Only process the first 3 batches
#     if batch_idx >= 3:
#         break

#     # Extract labels for the current batch
#     batch_labels = batch['label']

#     # Convert batch_labels to a list if it's not already (e.g., if it's a tensor)
#     if not isinstance(batch_labels, list):
#         batch_labels = batch_labels.tolist()

#     # Count the frequency of each label in this batch
#     label_counts = Counter(batch_labels)

#     # Print the label counts for this batch
#     print(f"Batch {batch_idx + 1} class counts:")
#     for label in range(30):  # Assuming classes are labeled from 0 to 29
#         print(f"  Class {label}: {label_counts[label]} instances")
#     print("-" * 30)  # Just a separator for clarity

# Train Teacher

## Prep s3 bucket

In [7]:
# Calling the function
model_name = 'resnet32x4_wider'
model_weight_path = 'weights/wider_teacher_resnet32x4_weights.pth'
num_class = 30
data_name = 'wider'  
batch_size = 32  
bucket_name = '210bucket'  

## Load in models

### resnet32x4_wider

In [8]:
# Instantiate the models
teacher_name = 'resnet32x4_wider'
teacher_model = models_package.__dict__[teacher_name](num_class=30)
teacher_model.fc = nn.Linear(teacher_model.fc.in_features, 30)

### resnet8x4_wider

In [9]:
student_name = 'resnet8x4_wider'
student_model = models_package.__dict__[student_name](num_class=30)
student_model.fc = nn.Linear(teacher_model.fc.in_features, 30)

In [10]:
# Optimizer and scheduler for the student model
student_optimizer = optim.SGD(student_model.parameters(), lr=learning_rate, momentum=momentum)
student_scheduler = torch.optim.lr_scheduler.StepLR(student_optimizer, step_size=step_size, gamma=gamma)

# Optimizer and scheduler for the teacher model
teacher_optimizer = optim.SGD(teacher_model.parameters(), lr=learning_rate, momentum=momentum)
teacher_scheduler = torch.optim.lr_scheduler.StepLR(teacher_optimizer, step_size=step_size, gamma=gamma)

criterion = nn.CrossEntropyLoss()
# Assuming the device is a CUDA device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# teacher_lr = best_LR_wider('resnet32x4_lr_wider_test', teacher_model, trainloader, criterion, teacher_optimizer, teacher_scheduler, device, num_epochs=3)
# teacher_lr

In [12]:
# student_lr = best_LR_wider('resnet8x4_lr_wider_test', student_model, trainloader, criterion, student_optimizer, student_scheduler, device, num_epochs=3)
# teacher_lr

In [13]:
teacher_lr = 0.0005953788301881531
student_lr = 0.0005953788301881531

## Train Leaderboard Teacher Models

In [ ]:
teacher_resnet32x4 = train_teacher_wider('resnet_32x4', teacher_model, trainloader, criterion, teacher_optimizer, teacher_scheduler, num_epochs=240, patience=5)


 31%|██████████████████████████▋                                                           | 100/323 [01:28<03:13,  1.15it/s]

[1, 100] loss: 6.326


 62%|█████████████████████████████████████████████████████▎                                | 200/323 [02:55<01:47,  1.15it/s]

[1, 200] loss: 3.349


 93%|███████████████████████████████████████████████████████████████████████████████▉      | 300/323 [04:22<00:20,  1.15it/s]

...

 62%|█████████████████████████████████████████████████████▎                                | 200/323 [02:54<01:47,  1.15it/s]

[54, 200] loss: 1.510


 93%|███████████████████████████████████████████████████████████████████████████████▉      | 300/323 [04:21<00:20,  1.15it/s]

[54, 300] loss: 1.566


  9%|███████▌                                                                               | 28/323 [00:25<04:16,  1.15it/s]